# Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import io
import requests
from datetime import datetime, timedelta
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime 
import dateutil.tz

In [ ]:
URL = 'https://www.worldometers.info/coronavirus/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
idx_incl_world = []
for idx, text in enumerate(soup.find_all('tr', class_='total_row_world')):
    if (str(text).find("World")!=-1):
        idx_incl_world.append(idx)
    

today_number = [txt.text for txt in soup.find_all('tr', class_='total_row')[idx_incl_world[0]].find_all('td')]
yesterday_number = [txt.text for txt in soup.find_all('tr', class_='total_row')[idx_incl_world[1]].find_all('td')]
now = datetime.now(dateutil.tz.tzutc())
print("Data Pulled @ {}".format(now))
print("\n<<RECENT UPDATE>>")
print(' Total Cases \t\t: {}\n New Cases \t\t: {}\n Total Deaths \t\t: {}\n New Deaths \t\t: {}\n Total Recovered \t: {}\n\
 Active Cases \t\t: {}\n Serious Critical \t: {}'.format(today_number[1],today_number[2],today_number[3],today_number[4]\
                                                 ,today_number[5],today_number[6],today_number[7]))
print("\n<<Yesterday>>")
print(' Total Cases \t\t: {}\n New Cases \t\t: {}\n Total Deaths \t\t: {}\n New Deaths \t\t: {}\n Total Recovered \t: {}\n\
 Active Cases \t\t: {}\n Serious Critical \t: {}'.format(yesterday_number[1],yesterday_number[2],yesterday_number[3],yesterday_number[4]\
                                                 ,yesterday_number[5],yesterday_number[6],yesterday_number[7]))

# job_elems = soup.find_all('script', type='text/javascript')
# for job_elem in job_elems:
#     if (job_elem.get_text().find('Highcharts.chart')!=-1):        
#         for line in job_elem.get_text().split('\n'):            
#             if(line.find(' Highcharts.chart(')!=-1):
#                 print(line)
#             if(line.find('data')!=-1):
#                 data = line.split('[')[1].split(']')[0].split(',')
#                 print(data)

# Load CSV Files
* Total Cases

In [ ]:
s0=requests.get('https://covid.ourworldindata.org/data/ecdc/total_cases.csv').content
total_cases = pd.read_csv(io.StringIO(s0.decode('utf-8'))).set_index('date')
total_cases.columns.name='Country/Region'
total_cases.tail()

* Total Deaths

In [ ]:
s1=requests.get('https://covid.ourworldindata.org/data/ecdc/total_deaths.csv').content
total_deaths = pd.read_csv(io.StringIO(s1.decode('utf-8'))).set_index('date')
total_deaths.columns.name='Country/Region'
total_deaths.tail()

* Total Recovered

In [ ]:
s2=requests.get('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_recovered_global.csv&filename=time_series_covid19_recovered_global.csv').content
total_recovered_raw = pd.read_csv(io.StringIO(s2.decode('utf-8')))
tr = total_recovered_raw.drop(['Lat','Long'],axis=1).groupby('Country/Region').sum().transpose()
tr = tr.rename(columns={"US":"United States",'Korea, South':'South Korea'})
tr_idx = tr.index
tr_idx_formatted = [(datetime.strptime(idx,'%m/%d/%y').strftime('%Y-%m-%d')) for idx in tr_idx]
tr = tr.set_index([pd.Series(tr_idx_formatted)])
tr_sum = tr.sum(axis=1).to_frame()
tr_sum.columns={"World"}
total_recovered = pd.concat([tr, tr_sum.reindex(tr.index)], axis=1)
total_recovered.index.names=['date']
total_recovered.tail()

# Select focused data
* List current top twenties countries and Singapore + Malaysia

In [ ]:
# today_str = datetime.utcnow().date().strftime('%Y-%m-%d')
# yesterday_str = (datetime.utcnow().date()-timedelta(days=1)).strftime('%Y-%m-%d')
tc_idx = total_cases.index
tc_latest_date = tc_idx[len(tc_idx)-1]
print("Latest Data Date : " , tc_latest_date)
total_cases = total_cases.sort_values(by=tc_latest_date,axis=1,ascending=False)
total_deaths = total_deaths.sort_values(by=tc_latest_date,axis=1,ascending=False)
Top_20 = list(total_cases.columns[0:21])
print('Top 20 countries are : ', Top_20[1:21])
Interested_countries = Top_20 + ['Singapore','Malaysia']

* Filter with only the interested countries

In [ ]:
IC_total_cases = total_cases[Interested_countries].fillna(method='ffill').fillna(0)
IC_total_deaths = total_deaths[Interested_countries].fillna(method='ffill').fillna(0)
IC_total_recovered = total_recovered[Interested_countries]

* Get growth rate tables

In [ ]:
IC_total_cases_inc = IC_total_cases.diff(axis=0).fillna(0)
IC_total_cases_inc.tail()

In [ ]:
# Moving average
IC_total_cases_inc_ma = IC_total_cases_inc.rolling(window=3).mean()
IC_total_cases_inc_ma.tail()

# Plot data

In [ ]:
for country in Interested_countries:
    plt.rcParams["figure.figsize"] = (20,5)
    fig, (ax1, ax2) = plt.subplots(2)        
    ax1.grid(color='b', linestyle='-.', linewidth=1)
    ax1.plot(list(IC_total_cases_inc_ma[country]))   
    ax2.set_yscale('log')
    ls_IC_total_cases = list(IC_total_cases[country])
    ax2.plot(ls_IC_total_cases)
    ax2.grid(color='b', linestyle='-.', linewidth=1)
    ax2.set_ylim([0,10**(round(np.log10(max(ls_IC_total_cases)))+1)])
    fig.suptitle(country)
    

# Plot table

In [ ]:
tr_idx = total_recovered.index
tr_latest_date = tr_idx[len(tr_idx)-1]
print(tr_latest_date)
last_day_rate = [int(IC_total_cases_inc[country][-1]) for country in Interested_countries]
last_3days_rate = [int(IC_total_cases_inc_ma[country][-1].mean()) for country in Interested_countries]
last_7days_rate = [int(IC_total_cases_inc[country][-7:-1].mean()) for country in Interested_countries]
TotalCase = [IC_total_cases[country][-1] for country in Interested_countries]
TotalDeath = [IC_total_deaths[country][-1] for country in Interested_countries]
TotalRecovered = list(IC_total_recovered.loc[tr_latest_date,:])
DeathRatio = [td/(tr+td)*100 for td, tr in zip(TotalDeath, TotalRecovered)]
Dropping = [l3<l7 for l3, l7 in zip(last_3days_rate, last_7days_rate)]


last_rate_dict = {'country':Interested_countries, 'Last day inc':last_day_rate,'Last 3 days average inc' :last_3days_rate, 
                  'Last 7 days inc':last_7days_rate, 'TotalCase': TotalCase, 'TotalRecovered':TotalRecovered,
                  'TotalDeath': TotalDeath, 'DeathRatio %': DeathRatio, 'Dropping':Dropping}
pd.DataFrame.from_dict(last_rate_dict).set_index('country')